In [7]:
import polars as pl
import pandas as pd

In [2]:
data_dir = "../data/03_primary"

In [40]:
schema = {
    'schema': pl.datatypes.Utf8,
    'name': pl.datatypes.Utf8,
    'value': pl.datatypes.Float64,
    'unit': pl.datatypes.Utf8,
    'instant': pl.datatypes.Utf8,
    'startdate': pl.datatypes.Utf8,
    'enddate': pl.datatypes.Utf8,
    'company': pl.datatypes.Utf8,
    'filing_date': pl.datatypes.Utf8,
}

df = pl.scan_csv(f"{data_dir}/combined_records_Accounts_Monthly_Data*.csv", schema=schema)


In [5]:
df.show_graph()

In [8]:
df1 = pd.read_csv("../data/03_primary/combined_records_Accounts_Monthly_Data-April2011.csv")

/tmp/ipykernel_5506/2468559740.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("../data/03_primary/combined_records_Accounts_Monthly_Data-April2011.csv")


In [89]:
schema_mapping = {}
name_mapping = {}
unit_mapping = {}

In [90]:
def update_mapping(unique_values, mapping):
	start_id = len(mapping)
	new_mappings = {value: i for i, value in enumerate(unique_values, start=start_id)}
	mapping.update(new_mappings)

def process_chunk(chunk):
	for column, mapping in zip(["schema", "name", "unit"],
							   [schema_mapping, name_mapping, unit_mapping]):
		unique_values = chunk[column].unique().to_numpy()
		unique_values = set(unique_values) - set(mapping.keys())
		update_mapping(unique_values, mapping)


	chunk = chunk.with_columns([pl.col("schema").map_dict(schema_mapping).alias("schema_id"),
				 pl.col("name").map_dict(name_mapping).alias('metric_id'),
				 pl.col("unit").map_dict(unit_mapping).alias('unit_id')])
	return chunk

# Process each CSV


In [91]:
import re
from datetime import datetime
def parse_date_from_filename(filename):
	# Use regular expression to extract the date part
	match = re.search(r'combined_records_Accounts_Monthly_Data-(\w+)(\d{4}).csv', filename)
	if match:
		month, year = match.groups()
		# Convert to a date object (assuming the first day of the month)
		try:
			v =  datetime.strptime(f'{month} {year}', '%B %Y')
		except ValueError:
			v=  datetime.strptime(f'January {year}', '%B %Y')
		return v


In [92]:
from pathlib import Path
filenames = [str(s) for s in Path(data_dir).glob("combined_records_Accounts_Monthly_Data*.csv")]
# Parse dates and sort
sorted_filenames = sorted(filenames, key=parse_date_from_filename)


In [93]:
file_paths = sorted_filenames
for file_path in file_paths:
    df = pl.read_csv(file_path, schema=schema)  # Using lazy evaluation
    processed_df = process_chunk(df)  # Adjust batch_size as needed
    # processed_df.write_csv(f"processed_{file_path}")
    facts = processed_df.select(pl.col("company").alias("company_id"), "schema_id", "metric_id", "unit_id", pl.col("instant").alias("instant_date"), pl.col("startdate").alias("start_date"), pl.col("enddate").alias("end_date"), "filing_date",  pl.col("value").alias("entry_value"))
    facts.write_database("account_entries","sqlite:///../data/03_primary/accounts.db", if_exists="append")


In [99]:
unit_df = pl.DataFrame({"unit_id": list(unit_mapping.values()), "unit_code": list(unit_mapping.keys())})
metric_df = pl.DataFrame({"metric_id": list(name_mapping.values()), "metric_code": list(name_mapping.keys())})
schema_df = pl.DataFrame({"schema_id": list(schema_mapping.values()), "schema_name": list(schema_mapping.keys())})

In [101]:
unit_df.write_database("units","sqlite:///../data/03_primary/accounts.db", if_exists="append")
metric_df.write_database("metrics","sqlite:///../data/03_primary/accounts.db", if_exists="append")
schema_df.write_database("schemas","sqlite:///../data/03_primary/accounts.db", if_exists="append")

In [102]:
sorted_filenames

['../data/03_primary/combined_records_Accounts_Monthly_Data-JanuaryToDecember2008.csv',
 '../data/03_primary/combined_records_Accounts_Monthly_Data-JanuaryToDecember2009.csv',
 '../data/03_primary/combined_records_Accounts_Monthly_Data-January2010.csv',
 '../data/03_primary/combined_records_Accounts_Monthly_Data-February2010.csv',
 '../data/03_primary/combined_records_Accounts_Monthly_Data-March2010.csv',
 '../data/03_primary/combined_records_Accounts_Monthly_Data-April2010.csv',
 '../data/03_primary/combined_records_Accounts_Monthly_Data-May2010.csv',
 '../data/03_primary/combined_records_Accounts_Monthly_Data-June2010.csv',
 '../data/03_primary/combined_records_Accounts_Monthly_Data-July2010.csv',
 '../data/03_primary/combined_records_Accounts_Monthly_Data-August2010.csv',
 '../data/03_primary/combined_records_Accounts_Monthly_Data-September2010.csv',
 '../data/03_primary/combined_records_Accounts_Monthly_Data-October2010.csv',
 '../data/03_primary/combined_records_Accounts_Monthly_D

In [94]:
schema_mapping

{'unknown': 0,
 'http://www.xbrl.org/uk/gaap/core/2009-09-01': 1,
 'http://www.xbrl.org/uk/reports/direp/2009-09-01': 2,
 'uk-gaap': 3,
 'uk-direp': 4,
 'uk-aurep': 5,
 'http://www.xbrl.org/uk/reports/aurep/2009-09-01': 6,
 'uk-gaap-pt': 7,
 'http://xbrl.frc.org.uk/reports/2014-09-01/aurep': 8,
 'http://xbrl.frc.org.uk/fr/2014-09-01/core': 9,
 'http://xbrl.frc.org.uk/reports/2014-09-01/direp': 10,
 'direp': 11,
 'aurep': 12,
 'core': 13,
 'http://xbrl.frc.org.uk/char/2016-01-01': 14,
 'http://xbrl.frc.org.uk/fr/2018-01-01/core': 15,
 'http://xbrl.frc.org.uk/reports/2018-01-01/direp': 16,
 'http://xbrl.frc.org.uk/reports/2018-01-01/aurep': 17,
 'http://xbrl.frc.org.uk/char/2018-01-01': 18,
 'http://xbrl.frc.org.uk/reports/2019-01-01/aurep': 19,
 'http://xbrl.frc.org.uk/reports/2019-01-01/direp': 20,
 'http://xbrl.frc.org.uk/fr/2019-01-01/core': 21,
 'http://xbrl.frc.org.uk/char/2019-01-01': 22,
 'http://xbrl.frc.org.uk/fr/2021-01-01/core': 23,
 'http://xbrl.frc.org.uk/reports/2021-01-01

In [62]:
df['schema'].apply(lambda x: schema_mapping[x])

/tmp/ipykernel_5506/2918807007.py:1: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  df['schema'].apply(lambda x: schema_mapping[x])
/tmp/ipykernel_5506/2918807007.py:1: PolarsInefficientMapWarning: 
Series.map_elements is significantly slower than the native series API.
Only use if you absolutely CANNOT implement your logic otherwise.
In this case, you can replace your `map_elements` with the following:
  - s.map_elements(lambda x: ...)
  + s.map_dict(schema_mapping)

  df['schema'].apply(lambda x: schema_mapping[x])


schema
i64
0
0
0
0
0
0
0
0
0


In [61]:
type(df['schema'])

polars.series.series.Series

In [51]:
processed_df.collect()

/tmp/ipykernel_5506/3659125393.py:14: PolarsInefficientMapWarning: 
Series.map_elements is significantly slower than the native series API.
Only use if you absolutely CANNOT implement your logic otherwise.
In this case, you can replace your `map_elements` with the following:
  - s.map_elements(lambda x: ...)
  + s.map_dict(mapping)

  chunk[column] = chunk[column].map_elements(lambda x: mapping[x])


ComputeError: User provided python function failed: TypeError: DataFrame object does not support `Series` assignment by index.

Use `DataFrame.with_columns`.

In [10]:
schemas = df.select("schema").group_by("schema").agg(pl.count("*")).collect(streaming=True).to_pandas()
names = df.select("name").group_by("name").agg(pl.count("*")).collect(streaming=True).to_pandas()
companies = df.select("company").group_by("company").agg(pl.count("*")).collect(streaming=True).to_pandas()

In [15]:
len(schemas)

31

In [9]:
df1[df1['startdate'].notna()]

,schema,name,value,unit,instant,startdate,enddate,company,filing_date
30,unknown,ParValueOrdinaryShare,5.0,iso4217:GBP,NaN,2010-01-01,2010-12-31,30977,2010-12-31
67,unknown,RateDepreciation,0.0,xbrli:pure,NaN,2010-01-01,2010-12-31,39757,2010-12-31
76,unknown,TangibleFixedAssetsAdditions,536.0,iso4217:GBP,NaN,2010-02-01,2011-01-31,44259,2011-01-31
79,unknown,TangibleFixedAssetsDepreciationChargeForPeriod,5871.0,iso4217:GBP,NaN,2010-02-01,2011-01-31,44259,2011-01-31
114,unknown,RateDepreciation,2.0,xbrli:pure,NaN,2010-02-01,2011-01-31,44259,2011-01-31
...,...,...,...,...,...,...,...,...,...
932506,unknown,TotalNominalValue,1.0,iso4217:GBP,NaN,2010-09-24,2011-03-31,SC385943,2011-03-31
932507,unknown,TotalConsideration,1.0,iso4217:GBP,NaN,2010-09-24,2011-03-31,SC385943,2011-03-31
932510,unknown,TangibleFixedAssetsAdditions,4095.0,iso4217:GBP,NaN,2010-10-14,2011-03-31,SC387068,2011-03-31
932512,unknown,TangibleFixedAssetsDepreciationChargeForPeriod,819.0,iso4217:GBP,NaN,2010-10-14,2011-03-31,SC387068,2011-03-31


In [20]:
df.approx_n_unique()

<LazyFrame [9 cols, {"schema": UInt32 … "filing_date": UInt32}] at 0x7F4C59A643D0>

In [6]:
agg = df.select("filing_date","company").group_by("filing_date").agg(pl.count("*"))

In [5]:
a = agg.collect(streaming=True)

In [7]:
agg2 = df.select("name").group_by("name").agg(pl.count("*")).collect(streaming=True)

In [11]:
agg2.count()

AttributeError: 'DataFrame' object has no attribute 'count'

In [12]:
import pandas as pd

In [13]:
a2 = pd.DataFrame(agg2)

In [15]:
a2

,0
0,DeferredTaxExpenseCreditRelatingToOriginationR...
1,ExpenseAllowancesDirectors
2,IntangibleFixedAssetsAmortisationOtherIncrease...
3,BankBorrowingsDebenturesDueWithinOneYear
4,ImpairmentReversalTangibleIntangibleFixedAssets
...,...
3222,BenefitsReceivedOrReceivablePerformanceInRepor...
3223,FurtherItemImpairmentLossReversalOnFinancialAs...
3224,TravelSubsistenceExpensesCostSales
3225,GainLossInCashFlowsFromChangeInContractAssets


In [17]:
a.set_index(0, inplace=True)

In [20]:
a.index = pd.to_datetime(a.index)

In [24]:
a.resample("Y").sum()

,1
0,
1999-12-31,18
2000-12-31,0
2001-12-31,33
2002-12-31,32
2003-12-31,80
2004-12-31,426
2005-12-31,5492
2006-12-31,83432
2007-12-31,4148665


In [25]:
a.sum()

1    709014430
dtype: object

In [2]:
units = df.select('name').unique().collect()

NameError: name 'df' is not defined

In [1]:
units

NameError: name 'units' is not defined

In [27]:
df.head(10).collect()

schema,name,value,unit,instant,startdate,enddate,company,filing_date
str,str,f64,str,str,str,str,str,str
"""unknown""","""Debtors""",25853.0,"""iso4217:GBP""","""2009-07-31""",null,null,"""00003006""","""2009-07-31"""
"""unknown""","""Debtors""",25853.0,"""iso4217:GBP""","""2008-07-31""",null,null,"""00003006""","""2009-07-31"""
"""unknown""","""CurrentAssets""",25853.0,"""iso4217:GBP""","""2009-07-31""",null,null,"""00003006""","""2009-07-31"""
"""unknown""","""CurrentAssets""",25853.0,"""iso4217:GBP""","""2008-07-31""",null,null,"""00003006""","""2009-07-31"""
"""unknown""","""NetCurrentAsse…",25853.0,"""iso4217:GBP""","""2009-07-31""",null,null,"""00003006""","""2009-07-31"""
"""unknown""","""NetCurrentAsse…",25853.0,"""iso4217:GBP""","""2008-07-31""",null,null,"""00003006""","""2009-07-31"""
"""unknown""","""TotalAssetsLes…",25853.0,"""iso4217:GBP""","""2009-07-31""",null,null,"""00003006""","""2009-07-31"""
"""unknown""","""TotalAssetsLes…",25853.0,"""iso4217:GBP""","""2008-07-31""",null,null,"""00003006""","""2009-07-31"""
"""unknown""","""NetAssetsLiabi…",25853.0,"""iso4217:GBP""","""2009-07-31""",null,null,"""00003006""","""2009-07-31"""
